<header>
   <p  style='font-size:36px;font-family:Arial; color:#F0F0F0; background-color: #00233c; padding-left: 20pt; padding-top: 20pt;padding-bottom: 10pt; padding-right: 20pt;'>
       HCA Healthcare Peer Facilities Clustering
  <br>
       <img id="teradata-logo" src="https://storage.googleapis.com/clearscape_analytics_demo_data/DEMO_Logo/teradata.svg" alt="Teradata" style="width: 125px; height: auto; margin-top: 20pt;">
    </p>
</header>


<p style = 'font-size:16px;font-family:Arial;color:#00233C'>A peer facility would be a facility of similar size, volume, and case mix. Peer Group Clusters allows the business to compare like-facilities together across several metrics, including revenue, expenses, outcomes, etc. In this demo, you will learn an efficient way to group acute care facilities into peer clusters with ClearScape Analytics. A peer facility would be a facility of similar size, volume, and case mix.</p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Grouping Columns:

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>
    <b>count_fte</b>: Total number of full-time employees as of the period end date. <br>
 <b>count_bed</b>: Total number of beds in service as of the period end date.<br>
 <b>count_admissions</b>: Total adjusted admissions as of the period end date. Adjusted Admissions is all inpatient admissions adjusted by an outpatient factor. Representative of total inpatient and outpatient volume.<br>
 <b>score_case_mix</b>: An indicator of inpatient acuity. Total DRG Weight / Total inpatient admissions as of the period end date.<br>
 <b>sum_net_revenue</b>: Total net revenue as of the period end date.<br>
 <b>sum_opex</b>: Total operating expenses as of the period end date.</p>
</p>


<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>1. Connect to Vantage</b></p>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>You will be prompted to provide the password. Enter your password, press Enter, then use down arrow to go to next cell.</p>

In [1]:
%connect local

Password: ········


Success: 'AzureCloud' connection established and activated for user 'db255096', with default database 'db255096'



<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>2. Data Preparation</b></p>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>

In [2]:
select top 5* from features

,entity_id,entity_name,special_facility,Exclude_Facility,measure_dt,count_fte,count_beds,count_admissions,score_case_mix,sum_net_revenue,sum_opex
1,26109,METHODIST TEXSAN HOSPITAL,0,0,2023-12-31,518,120,10819,1.88,165664371,123692956
2,27400,THE MEDICAL CENTER AURORA,0,0,2023-12-31,1465,389,30626,1.78,420474649,343874458
3,8298,CORPUS CHRISTI MEDICAL CENTER,0,0,2023-12-31,1656,421,39350,1.51,490845548,376009568
4,25325,SPOTSYLVANIA REGIONAL MEDICAL CENTER,0,0,2023-12-31,522,127,19552,1.32,182678920,124783461
5,31767,GREENVIEW REGIONAL HOSPITAL,0,0,2023-12-31,438,118,14437,1.54,143532991,109108076


<p style = 'font-size:16px;font-family:Arial;color:#00233C'><b><a href = 'https://docs.teradata.com/r/Enterprise_IntelliFlex_VMware/Database-Analytic-Functions/Feature-Engineering-Transform-Functions/Antiselect'>Antiselect</a> </b> returns all columns except those specified in the Exclude syntax element.</p>

In [6]:
drop view v_features;

Success: 15 rows affected

In [7]:
--Create View to that can be used to filter out special or excluded facilities with missing
create volatile table v_features AS (

select * from Antiselect (
        on (SELECT hf.*
        FROM features hf
        JOIN original_cluster hu ON hf.entity_id = hu.entity_id
        WHERE hu.cluster_id IS NOT NULL
        AND hf.Exclude_Facility < 1
        AND hf.special_facility < 1)  
        USING
  Exclude ('special_facility', 'Exclude_Facility')
) as dt)with data ON COMMIT PRESERVE ROWS;

Success: 0 rows affected

In [8]:
select top 5* from v_features;

,entity_id,entity_name,measure_dt,count_fte,count_beds,count_admissions,score_case_mix,sum_net_revenue,sum_opex
1,32905,PARKLAND MEDICAL CENTER,2023-12-31,430,85,12513,1.44,159773914,113260281
2,30997,HCA FLORIDA OAK HILL HOSPITAL,2023-12-31,1275,350,32348,1.52,370906368,246575422
3,37938,HCA FLORIDA LAKE CITY HOSPITAL,2023-12-31,565,113,24559,1.38,168408021,103403864
4,30932,HCA FLORIDA LAWNWOOD HOSPITAL,2023-12-31,1637,432,34466,1.72,531541178,357143367
5,31003,DOCTORS HOSPITAL OF AUGUSTA,2023-12-31,1501,331,25122,1.88,493507285,365612369


<p style = 'font-size:16px;font-family:Arial;color:#00233C'> <b> 2.1 Checking data demographics  </b> </p>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'><b><a href = 'https://docs.teradata.com/r/Enterprise_IntelliFlex_VMware/Database-Analytic-Functions/Data-Exploration-Functions/TD_ColumnSummary'>TD_ColumnSummary</a> </b>function  displays Column name, datatype and other demographics like count of NULLs etc for each specified input table column</p>

In [9]:
SELECT * FROM TD_ColumnSummary (
 ON v_features AS InputTable
 USING
 TargetColumns ('[:]')
) AS dt;

,ColumnName,Datatype,NonNullCount,NullCount,BlankCount,ZeroCount,PositiveCount,NegativeCount,NullPercentage,NonNullPercentage
1,count_fte,INTEGER,165,2,,1,164,0,1.1976047904191616,98.80239520958084
2,sum_net_revenue,BIGINT,165,2,,0,165,0,1.1976047904191616,98.80239520958084
3,count_admissions,INTEGER,165,2,,2,163,0,1.1976047904191616,98.80239520958084
4,sum_opex,BIGINT,167,0,,2,165,0,0,100
5,measure_dt,DATE,167,0,,,,,0,100
6,score_case_mix,"DECIMAL(3,2)",165,2,,2,163,0,1.1976047904191616,98.80239520958084
7,count_beds,INTEGER,165,2,,2,163,0,1.1976047904191616,98.80239520958084
8,entity_name,VARCHAR(255) CHARACTER SET LATIN,167,0,0,,,,0,100
9,entity_id,INTEGER,167,0,,0,167,0,0,100


<hr style="height:1px;border:none;background-color:#00233C;">
<p style = 'font-size:16px;font-family:Arial;color:#00233C'><b>2.2 Gather statistics using TD_UnivariateStatistics</b></p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Univariate analysis is the simplest form of analyzing data. Its significant purpose is to describe; It takes data, summarizes it, and finds patterns in the data.
    <br>
    <a href = 'https://docs.teradata.com/search/all?query=TD_UnivariateStatistics&content-lang=en-US'>TD_UnivariateStatistics</a> displays descriptive statistics for each specified numeric input table column.
</p>

In [10]:
SELECT *
FROM TD_UnivariateStatistics (
ON v_features  AS InputTable
USING
TargetColumns('count_fte' , 'count_beds' ,'count_admissions' ,'score_case_mix','sum_net_revenue' ,'sum_opex')
STATS( 
      'MEAN',
      'MEDIAN',
      'MODE',           
      'ZERO VALUES COUNT',
      'PERCENTILES',
      'MINIMUM',
      'MAXIMUM'
        )
        ) as dt order by 1,2;

,ATTRIBUTE,StatName,StatValue
1,count_admissions,MAXIMUM,78804
2,count_admissions,MEAN,21993.951515151515
3,count_admissions,MEDIAN,19048
4,count_admissions,MINIMUM,0
5,count_admissions,MODE,0
6,count_admissions,PERCENTILES(1),821.1200000000001
7,count_admissions,PERCENTILES(10),8235.6
8,count_admissions,PERCENTILES(25),11554
9,count_admissions,PERCENTILES(5),5561.8
10,count_admissions,PERCENTILES(50),19048


<hr style="height:1px;border:none;background-color:#00233C;">
<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>2.3 Scale Transform </b></p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Many machine learning models, especially those that rely on distance calculations (e.g., K-Means, K-Nearest Neighbors, Support Vector Machines, and Principal Component Analysis), assume that all features contribute equally. If one feature has a much larger scale than others, it can dominate the model’s behavior.

<p style = 'font-size:16px;font-family:Arial;color:#00233C'><a href = 'https://docs.teradata.com/r/Enterprise_IntelliFlex_VMware/Database-Analytic-Functions/Feature-Engineering-Transform-Functions/TD_ScaleFit'>TD_ScaleFit</a> outputs a table of statistics to input to TD_ScaleTransform, which scales specified input table columns. TD_ScaleFit accepts input data in dense and sparse format.
    <br>
    <a href = 'https://docs.teradata.com/search/all?query=TD_UnivariateStatistics&content-lang=en-US'>TD_ScaleTransform</a> scales specified input table columns, using TD_ScaleFit output. TD_ScaleTransform accepts input data in dense and sparse format.
</p>


In [11]:
drop table features_scaled;

Success: 20 rows affected

In [13]:
--Calculate Scaling parameters
create volatile table features_scaled as (
SELECT * FROM TD_ScaleFit(
    ON v_features AS InputTable
    --OUT TABLE OutputTable(features_scaled)
    USING
        TargetColumns('count_fte' , 'count_beds' ,'count_admissions' ,'score_case_mix','sum_net_revenue' ,'sum_opex')
        ScaleMethod('range')
        MissValue('zero')
) as dt)with data ON COMMIT PRESERVE ROWS;


Success: 0 rows affected

In [15]:
drop table v_features_scaled

Success: 21 rows affected

In [16]:
create volatile table v_features_scaled as (

SELECT hf.entity_id, hs.count_fte, hs.count_beds, hs.count_admissions, hs.score_case_mix, hs.sum_net_revenue, hs.sum_opex 
FROM TD_scaleTransform (
  ON v_features AS InputTable
  ON features_scaled AS FitTable DIMENSION
  USING
  Accumulate ('entity_id')
) AS hs
	JOIN v_features hf on (hf.entity_id = hs.entity_id)
) with data ON COMMIT PRESERVE ROWS;

Success: 0 rows affected

In [17]:
select * from v_features_scaled

,entity_id,count_fte,count_beds,count_admissions,score_case_mix,sum_net_revenue,sum_opex
1,39233,0.4401382374722291,0.4848851269649335,0.4606999644688087,0.5818181818181819,0.41647986137737164,0.40546270812027785
2,26890,0.04196494692668477,0.06408706166868199,0.07939952286685956,0.49090909090909096,0.029989674724945573,0.03676272841133463
3,9727,0.42754875339422366,0.5344619105199516,0.35595908837114865,0.6872727272727273,0.43068909687238927,0.4853200339905768
4,34293,0.31695877561096025,0.34582829504232165,0.2692502918633572,0.7309090909090908,0.2866084938819792,0.28227387360565587
5,30941,0.27993088126388543,0.3857315598548972,0.3912745545911375,0.5236363636363636,0.23687538372748143,0.23057050441005175
6,26920,0.3376943964453221,0.37122128174123337,0.3171285721536978,0.5672727272727273,0.30708693037520784,0.33625807066277325
7,9722,0.2979511231794619,0.32769044740024184,0.3025100248718339,0.5927272727272727,0.3578935748183795,0.3089273232711425
8,31054,0.15971365095038262,0.16807738814993953,0.23383330795391097,0.48363636363636364,0.12179145643197026,0.13036132611704032
9,27620,0.05726980992347568,0.03506650544135429,0.12800111669458403,0.45454545454545453,0.042954711393487215,0.05234215711302352
10,26110,0.3268328807701802,0.4026602176541717,0.3725952997309781,0.52,0.275964822157525,0.2909644373136267


<hr style="height:1px;border:none;background-color:#00233C;">
<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b> 3. Kmeans Clustering</b></p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Kmeans is an unsupervised machine learning algorithm used for clustering. It partitions a dataset into K clusters, where each data point belongs to the cluster with the nearest mean (centroid), in this case, the HCA healthcare facilities will be grouped into K cluters using Kmeans clustering method.</p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'><a href = 'https://docs.teradata.com/r/Enterprise_IntelliFlex_VMware/Database-Analytic-Functions/Model-Training-Functions/TD_KMeans'>TD_KMeans</a> The k-means algorithm groups a set of observations into k clusters in which each observation belongs to the cluster with the nearest mean (cluster centers or cluster centroid).
    <br>
    <a href = 'https://docs.teradata.com/r/Enterprise_IntelliFlex_VMware/Database-Analytic-Functions/Model-Scoring-Functions/TD_KMeansPredict'>TD_KMeansPredict</a>  uses the k-means algorithm to predict the target class of unseen or new data.
</p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>This algorithm minimizes the objective function, that is, the total Euclidean distance of all data points from the center of the cluster as follows:</p>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>
    1.Specify or randomly select k initial cluster centroids. <br>
    2.Assign each data point to the cluster that has the closest centroid.<br>
    3.Recalculate the positions of the k centroids.<br>
    4.Repeat steps 2 and 3 until the centroids no longer move.
</p>



In [18]:
drop table KMeans_Model;

Success: 24 rows affected

In [19]:
--perform KMeans Clustering
Select * from TD_KMeans (
    ON v_features_scaled as InputTable
    OUT TABLE ModelTable(KMeans_Model) --KMeans_Model table is used to save the weights of the model
    USING
        IdColumn('entity_id')
        TargetColumns( 'count_fte' , 'count_beds' ,'count_admissions' ,'score_case_mix','sum_net_revenue' ,'sum_opex')
        InitialCentroidsMethod('kmeans++')
        StopThreshold(0.001)
        NumClusters(5)
        --Seed(0)
        MaxIterNum(100)
)as dt;


Success: 0 rows affected

,td_clusterid_kmeans,count_fte,count_beds,count_admissions,score_case_mix,sum_net_revenue,sum_opex,td_size_kmeans,td_withinss_kmeans,entity_id,td_modelinfo_kmeans
1,0,0.38242409281658846,0.46571083088616333,0.41366500612731766,0.6214025974025974,0.3856629835259401,0.3757373682792589,35,0.9395282497611674,,
2,1,0.08832739711535065,0.10559682155812745,0.13466448403634337,0.4827012987012987,0.07499661856003924,0.07848927590817936,70,2.046354238326643,,
3,2,0.9983954579116268,0.996372430471584,0.9172440485254556,0.6763636363636363,0.9990012828489401,0.9881377015616567,2,0.014117770885253123,,
4,3,0.24302641323130098,0.2929141475211609,0.2779838586873763,0.6045090909090909,0.22680209513678498,0.2288815720353529,50,1.2514009656441658,,
5,4,0.6209331029375462,0.7500604594921403,0.6412517131110096,0.6352727272727272,0.6184375034032411,0.6092990451033049,10,0.7359083241183697,,
6,,,,,,,,,,,Converged : True
7,,,,,,,,,,,Number of Iterations : 13
8,,,,,,,,,,,Number of Clusters : 5
9,,,,,,,,,,,Total_WithinSS : 4.98730954873560E+00
10,,,,,,,,,,,Between_SS : 2.66069968833987E+01


In [21]:
drop table clusters

Success: 17 rows affected

In [22]:
create volatile table clusters as (

SELECT entity_id, td_clusterid_kmeans as cluster_id, CURRENT_DATE() as cluter_date 
FROM TD_KMeansPredict (
    ON v_features_scaled AS InputTable
    ON KMeans_Model as ModelTable DIMENSION
    USING
        OutputDistance('false')
) as dt) with data ON COMMIT PRESERVE ROWS;

Success: 0 rows affected

In [23]:
select * from clusters

,entity_id,cluster_id,cluter_date
1,39233,0,2025-03-21
2,26890,1,2025-03-21
3,9727,0,2025-03-21
4,34293,3,2025-03-21
5,30941,3,2025-03-21
6,26920,0,2025-03-21
7,9722,3,2025-03-21
8,31054,1,2025-03-21
9,27620,1,2025-03-21
10,26110,0,2025-03-21


In [25]:
drop view final_tab;

Success: 16 rows affected

In [26]:
--Optional query to Join Original Clusters with Teradata
create volatile table final_tab as (
SELECT DISTINCT 
    hc.cluster_id AS Teradata_Cluster,
    original_cluster.cluster_id as Original_Cluster,
    hf.entity_id,      -- Include any other non-aggregated columns you want
    hf.entity_name,
    hf.count_admissions,
    hf.count_beds,
    hf.count_fte, 
    hf.sum_net_revenue, 
    hf.sum_opex,
    hf.score_case_mix
 FROM 
    v_features hf
JOIN 
    clusters hc ON hf.entity_id = hc.entity_id
LEFT JOIN original_cluster
    ON hf.entity_id = original_cluster.entity_id
)with data ON COMMIT PRESERVE ROWS;

Success: 0 rows affected

In [27]:
select * from final_tab

,Teradata_Cluster,Original_Cluster,entity_id,entity_name,count_admissions,count_beds,count_fte,sum_net_revenue,sum_opex,score_case_mix
1,3,2,3132,METHODIST HOSPITAL STONE OAK,26527,241,1100,363265494,262107239,1.61
2,2,1,27610,MISSION HOSPITAL,78804,821,4051,1368498651,987982056,1.88
3,4,1,26960,MEMORIAL HEALTH UMC SAVANNAH,53135,599,2655,769548229,607208617,1.88
4,1,0,6678,ST DAVID'S REHAB,,,,,0,
5,0,2,26920,HCA HOUSTON NORTHWEST,24991,307,1368,423010643,332216940,1.56
6,3,2,8385,REGIONAL MEDICAL CENTER OF SAN JOSE,15363,192,1054,345167998,365409909,1.83
7,2,1,39385,METHODIST HOSPITAL,65761,827,4038,1371232053,964542580,1.84
8,4,1,31608,WESLEY MEDICAL CENTER,43153,506,2108,745045705,472300442,1.69
9,1,0,9729,RESEARCH PSYCHIATRIC CENTER,,,,,0,
10,0,2,30913,HCA FLORIDA ORANGE PARK HOSPITAL,30231,348,1386,428004422,283528692,1.67


<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b> 5. Model Accuracy Check</b></p>


<p style = 'font-size:16px;font-family:Arial;color:#00233C'><a href = 'https://docs.teradata.com/r/Enterprise_IntelliFlex_VMware/Database-Analytic-Functions/Model-Evaluation-Functions/TD_Silhouette'>TD_Silhouette</a> function is a method of interpretation and validation of consistency within clusters of data. The function determines how well the data is clustered among clusters.


<p style = 'font-size:16px;font-family:Arial;color:#00233C'>
Silhouette Score measures how well each point fits in its assigned cluster (range: -1 to 1).</p>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>
1 → Perfect clustering <br>
0 → Overlapping clusters <br>
-1 → Poor clustering (misclassified points) 
</p>

In [28]:

SELECT * FROM TD_Silhouette(
ON final_tab as InputTable
USING
 IdColumn('entity_id')
 ClusterIdColumn('Teradata_Cluster')
 TargetColumns('count_fte' , 'count_beds' ,'count_admissions' ,'score_case_mix','sum_net_revenue' ,'sum_opex')
 OutputType('CLUSTER_SCORES')
) as dt;

,Teradata_Cluster,silhouette_score
1,3,0.39522100358019274
2,2,0.9634131617466692
3,4,0.20367729590261954
4,1,0.6788511378554553
5,0,0.3334679240214497


<p style = 'font-size:16px;font-family:Arial;color:#00233C'>

Clusters 2 and 1 have very high silhouette scores (0.95), meaning they are well-separated and compact.<br>
Clusters 4 and 5 have relatively low scores (0.28 - 0.32), suggesting overlapping clusters. <br>
Overall average is moderate (0.4), indicating decent but not perfect clustering.

In [29]:
SELECT * FROM TD_Silhouette(
ON final_tab as InputTable
USING
 IdColumn('entity_id')
 ClusterIdColumn('Original_Cluster')
 TargetColumns('count_fte' , 'count_beds' ,'count_admissions' ,'score_case_mix','sum_net_revenue' ,'sum_opex')
 OutputType('CLUSTER_SCORES')
) as dt;

,Original_Cluster,silhouette_score
1,3,0.3157027259077065
2,5,0.20985513672869832
3,2,0.3055782736664925
4,4,-0.2567482861511197
5,1,0.28939898746592924


Cluster 4 has a negative silhouette score (0.256) → likely misclassified or overlapping significantly with other clusters. <br>
The highest score is  around 0.31, which is quite low overall, indicating poor cluster separation. <br>
Overall, the second model is weaker than the first one, as most clusters have lower scores compared to the first model.


Conclusions

1. The first K-Means model performs better overall (higher silhouette scores). <br>
2. Some clusters are well-formed (Cluster 1 and 2 with scores 0.95).<br>
3. However, Clusters 4 and 5 still need improvement (low scores).<br>
4. The second K-Means model has lower silhouette scores, with one cluster being misclassified (negative score).<br>
5. Indicates overlapping clusters and poor separation.<br>
6. Needs improvement in feature scaling, number of clusters (K), or different distance metrics.

Recommendation

1. Reevaluate the choice of K: Use the Elbow Method to determine the optimal number of clusters. <br>
2. Try different distance metrics: Instead of Euclidean distance, test cosine similarity or Manhattan distance.<br>
3. Feature Engineering: Remove redundant features or apply Principal Component Analysis (PCA) to improve separability.<br>
4. Check for outliers: Outliers can distort clustering results. Consider using DBSCAN if data has noise.